In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.interpolate import interp1d
from tqdm import tqdm
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv('/content/modified_ais_data.csv')
df.head()

,BaseDateTime,MMSI,IMO,LAT,LON,SOG,COG
0,2022-03-31T00:00:01,671226100,IMO9221322,25.77626,-80.20320,3.2,143.7
1,2022-03-31T00:00:06,367452810,IMO9602344,29.32824,-94.77391,2.6,319.2
2,2022-03-31T00:00:09,366919770,IMO9253583,48.74428,-122.49504,0.0,210.3
3,2022-03-31T00:00:00,311000966,IMO8916607,18.55833,-66.47910,17.5,274.5
4,2022-03-31T00:00:01,219028420,IMO9411305,28.66703,-93.59339,9.5,129.4


In [6]:
df['MMSI'].unique()

array([671226100, 367452810, 366919770, ..., 636092799, 441563000,
       477203400])

In [9]:
sorted = df['MMSI'].sort_values().unique()

In [10]:
sorted

array([      111,   1056261, 205086000, ..., 671960000, 671969100,
       710023040])

In [3]:
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm import tqdm

def sliding_window_segmentation(data, window_size, step_size, mmsi):
    segments = []
    for start in range(0, len(data) - window_size + 1, step_size):
        window = data.iloc[start:start + window_size].copy()
        window['MMSI'] = mmsi
        segments.append(window)
    return segments

def lat_lon_range(df_file):
    min_lat, max_lat = df_file['LAT'].min(), df_file['LAT'].max()
    min_lon, max_lon = df_file['LON'].min(), df_file['LON'].max()
    min_lat, max_lat, min_lon, max_lon = (
        int(np.floor(min_lat)),
        int(np.ceil(max_lat)),
        int(np.floor(min_lon)),
        int(np.ceil(max_lon)),
    )
    return min_lat, max_lat, min_lon, max_lon

def prepare_ais_data_without_interpolation(
    data, bounding_box, window_size, step_size
):
    min_lat, max_lat, min_lon, max_lon = bounding_box

    filtered_data = data[
        (data['LAT'] >= min_lat) & (data['LAT'] <= max_lat) &
        (data['LON'] >= min_lon) & (data['LON'] <= max_lon)
    ]

    grouped = filtered_data.groupby('MMSI')
    segments = []
    segment_id = 0

    for mmsi, group in tqdm(grouped, desc="Processing MMSI", unit="MMSI"):
        group = group.sort_values(by='BaseDateTime')
        group['BaseDateTime'] = pd.to_datetime(group['BaseDateTime'])

        # Directly use the existing data without interpolation
        resampled_traj = group[['BaseDateTime', 'LAT', 'LON', 'SOG', 'COG']].copy()

        trajectory_segments = sliding_window_segmentation(resampled_traj, window_size, step_size, mmsi)

        for segment in trajectory_segments:
            segment['SegmentID'] = segment_id
            segments.append(segment)
            segment_id += 1

    combined_df = pd.concat(segments, ignore_index=True)
    return combined_df

# Example usage
if __name__ == "__main__":
    df = pd.read_csv("/content/modified_ais_data.csv")[:30000]  # Adjust to your file path

    # Define bounding box from the dataset
    min_lat, max_lat, min_lon, max_lon = lat_lon_range(df)
    bounding_box = (min_lat, max_lat, min_lon, max_lon)

    window_size = 10
    step_size = 5

    segmented_trajectories_df = prepare_ais_data_without_interpolation(
        df, bounding_box, window_size, step_size
    )

    segmented_trajectories_df.to_csv("segmented_trajectories.csv", index=False)


Processing MMSI: 100%|██████████| 2089/2089 [00:13<00:00, 159.11MMSI/s]


In [4]:
segmented_trajectories_df.head()

,BaseDateTime,LAT,LON,SOG,COG,MMSI,SegmentID
0,2022-03-31 00:02:32,27.35372,-94.62546,0.4,228.6,111,0
1,2022-03-31 00:05:35,27.35372,-94.62550,0.6,219.8,111,0
2,2022-03-31 00:08:34,27.35377,-94.62556,0.2,221.7,111,0
3,2022-03-31 00:11:31,27.35380,-94.62557,0.3,105.0,111,0
4,2022-03-31 00:14:33,27.35365,-94.62542,0.3,173.4,111,0


In [7]:
import geopandas as gpd
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from shapely.geometry import Point as ShapelyPoint
from tqdm import tqdm
from math import atan2, radians, degrees, sin, cos

def sliding_window_segmentation(data, window_size, step_size, mmsi):
    segments = []
    for start in range(0, len(data) - window_size + 1, step_size):
        window = data.iloc[start:start + window_size].copy()
        window['MMSI'] = mmsi
        segments.append(window)
    return segments

def lat_lon_range(df_file):
    min_lat, max_lat = df_file['LAT'].min(), df_file['LAT'].max()
    min_lon, max_lon = df_file['LON'].min(), df_file['LON'].max()
    min_lat, max_lat, min_lon, max_lon = (
        int(np.floor(min_lat)),
        int(np.ceil(max_lat)),
        int(np.floor(min_lon)),
        int(np.ceil(max_lon)),
    )
    return min_lat, max_lat, min_lon, max_lon

def calculate_initial_bearing(start_lat, start_lon, end_lat, end_lon):
    start_lat, start_lon, end_lat, end_lon = map(radians, [start_lat, start_lon, end_lat, end_lon])
    delta_lon = end_lon - start_lon
    x = sin(delta_lon) * cos(end_lat)
    y = cos(start_lat) * sin(end_lat) - sin(start_lat) * cos(end_lat) * cos(delta_lon)
    bearing = atan2(x, y)
    return (degrees(bearing) + 360) % 360

def geodesic_interpolation(start_lat, start_lon, end_lat, end_lon, num_points, start_time, end_time, water_mask):
    start_point = ShapelyPoint(start_lon, start_lat)
    total_distance = geodesic((start_lat, start_lon), (end_lat, end_lon)).meters
    bearing = calculate_initial_bearing(start_lat, start_lon, end_lat, end_lon)

    lats, lons, times = [], [], []
    spatial_index = water_mask.sindex
    buffered_water_mask = water_mask.buffer(0.01)

    timestamps = pd.date_range(start=start_time, end=end_time, periods=num_points)

    for frac, timestamp in zip(np.linspace(0, 1, num_points), timestamps):
        distance_to_point = total_distance * frac
        interpolated_point = geodesic(meters=distance_to_point).destination((start_lat, start_lon), bearing)
        lat, lon = interpolated_point.latitude, interpolated_point.longitude

        point = ShapelyPoint(lon, lat)
        possible_matches_index = list(spatial_index.intersection(point.bounds))
        possible_matches = buffered_water_mask.iloc[possible_matches_index]

        if not possible_matches.contains(point).any():
            print(f"Skipping interpolated point on land or near boundary: ({lat}, {lon})")
            continue

        lats.append(lat)
        lons.append(lon)
        times.append(timestamp)

    return lats, lons, times

def prepare_ais_data_with_geodesic_interpolation(
    data, bounding_box, window_size, step_size, water_mask, min_resample_interval=3600
):
    min_lat, max_lat, min_lon, max_lon = bounding_box

    filtered_data = data[
        (data['LAT'] >= min_lat) & (data['LAT'] <= max_lat) &
        (data['LON'] >= min_lon) & (data['LON'] <= max_lon)
    ]

    grouped = filtered_data.groupby('MMSI')
    segments = []
    segment_id = 0

    for mmsi, group in tqdm(grouped, desc="Processing MMSI", unit="MMSI"):
        group = group.sort_values(by='BaseDateTime')
        group['BaseDateTime'] = pd.to_datetime(group['BaseDateTime'])

        resampled_traj = []

        for i in range(len(group) - 1):
            start_row = group.iloc[i]
            end_row = group.iloc[i + 1]

            time_diff = (end_row['BaseDateTime'] - start_row['BaseDateTime']).total_seconds()

            if time_diff < min_resample_interval:
                print(f"Skipping interpolation for gap < {min_resample_interval} minutes between {start_row['BaseDateTime']} and {end_row['BaseDateTime']}")
                resampled_traj.append((start_row['BaseDateTime'], start_row['LAT'], start_row['LON'], start_row['SOG'], start_row['COG']))
                continue

            num_points = int(time_diff / min_resample_interval) + 1
            print(f"Performing interpolation for gap >= {min_resample_interval} minutes between {start_row['BaseDateTime']} and {end_row['BaseDateTime']} with {num_points} points")

            lats, lons, times = geodesic_interpolation(
                start_row['LAT'], start_row['LON'],
                end_row['LAT'], end_row['LON'],
                num_points, start_row['BaseDateTime'], end_row['BaseDateTime'], water_mask
            )

            sog_interp = np.linspace(start_row['SOG'], end_row['SOG'], len(lats))
            cog_interp = np.linspace(start_row['COG'], end_row['COG'], len(lats))

            resampled_traj.extend(zip(times, lats, lons, sog_interp, cog_interp))

        resampled_traj.append((end_row['BaseDateTime'], end_row['LAT'], end_row['LON'], end_row['SOG'], end_row['COG']))
        resampled_df = pd.DataFrame(resampled_traj, columns=['BaseDateTime', 'LAT', 'LON', 'SOG', 'COG'])

        trajectory_segments = sliding_window_segmentation(resampled_df, window_size, step_size, mmsi)

        for segment in trajectory_segments:
            segment['SegmentID'] = segment_id
            segments.append(segment)
            segment_id += 1

        # Append resampled data to the original group
        group = pd.concat([group, resampled_df]).sort_values(by='BaseDateTime')

    combined_df = pd.concat(segments, ignore_index=True)
    return combined_df

# Example usage
if __name__ == "__main__":
    df = df[:30000]
    # water_mask = gpd.read_file("/path/to/world_shapefile.shp")
    water_url = "https://naciscdn.org/naturalearth/110m/physical/ne_110m_geography_marine_polys.zip"
    water_mask = gpd.read_file(water_url)

    min_lat, max_lat, min_lon, max_lon = lat_lon_range(df)
    bounding_box = (min_lat, max_lat, min_lon, max_lon)

    window_size = 10
    step_size = 5

    segmented_trajectories_df = prepare_ais_data_with_geodesic_interpolation(
        df, bounding_box, window_size, step_size, water_mask, min_resample_interval=900
    )

    segmented_trajectories_df.to_csv("segmented_trajectories.csv", index=False)


Processing MMSI:   0%|          | 8/2089 [00:00<00:33, 62.61MMSI/s]

Skipping interpolation for gap < 900 minutes between 2022-03-31 00:02:32 and 2022-03-31 00:05:35
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:05:35 and 2022-03-31 00:08:34
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:08:34 and 2022-03-31 00:11:31
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:11:31 and 2022-03-31 00:14:33
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:14:33 and 2022-03-31 00:17:33
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:17:33 and 2022-03-31 00:20:32
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:20:32 and 2022-03-31 00:23:32
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:23:32 and 2022-03-31 00:29:36
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:29:36 and 2022-03-31 00:32:33
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:00:17 and 2022-03-31 00:01:18
Skipping interpolation for gap

Processing MMSI:   1%|          | 15/2089 [00:00<00:32, 63.16MMSI/s]

Skipping interpolation for gap < 900 minutes between 2022-03-31 00:01:21 and 2022-03-31 00:02:30
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:02:30 and 2022-03-31 00:03:40
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:03:40 and 2022-03-31 00:04:50
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:04:50 and 2022-03-31 00:05:59
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:05:59 and 2022-03-31 00:07:11
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:07:11 and 2022-03-31 00:08:21
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:08:21 and 2022-03-31 00:09:31
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:09:31 and 2022-03-31 00:10:41
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:10:41 and 2022-03-31 00:11:51
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:11:51 and 2022-03-31 00:13:01
Skipping interpolation for gap

Processing MMSI:   2%|▏         | 32/2089 [00:00<00:38, 53.99MMSI/s]

...


Skipping interpolation for gap < 900 minutes between 2022-03-31 00:28:14 and 2022-03-31 00:30:26
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:30:26 and 2022-03-31 00:31:32
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:02:17 and 2022-03-31 00:05:22
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:05:22 and 2022-03-31 00:08:20
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:08:20 and 2022-03-31 00:11:23
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:11:23 and 2022-03-31 00:14:19
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:14:19 and 2022-03-31 00:17:17
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:17:17 and 2022-03-31 00:20:19
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:20:19 and 2022-03-31 00:23:19
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:23:19 and 2022-03-31 00:26:22
Skipping interpolation for ga

Processing MMSI: 100%|█████████▉| 2081/2089 [01:09<00:00, 30.08MMSI/s]


Skipping interpolation for gap < 900 minutes between 2022-03-31 00:08:43 and 2022-03-31 00:11:42
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:11:42 and 2022-03-31 00:14:39
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:14:39 and 2022-03-31 00:17:43
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:17:43 and 2022-03-31 00:23:43
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:23:43 and 2022-03-31 00:29:43
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:00:00 and 2022-03-31 00:01:10
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:01:10 and 2022-03-31 00:02:21
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:02:21 and 2022-03-31 00:03:30
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:03:30 and 2022-03-31 00:04:31
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:04:31 and 2022-03-31 00:05:40
Skipping interpolation for ga

Processing MMSI: 100%|██████████| 2089/2089 [01:10<00:00, 29.82MMSI/s]


Skipping interpolation for gap < 900 minutes between 2022-03-31 00:00:14 and 2022-03-31 00:01:54
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:01:54 and 2022-03-31 00:03:04
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:03:04 and 2022-03-31 00:04:05
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:04:05 and 2022-03-31 00:05:14
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:05:14 and 2022-03-31 00:06:24
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:06:24 and 2022-03-31 00:07:54
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:07:54 and 2022-03-31 00:09:03
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:09:03 and 2022-03-31 00:10:24
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:10:24 and 2022-03-31 00:11:53
Skipping interpolation for gap < 900 minutes between 2022-03-31 00:11:53 and 2022-03-31 00:13:03
Skipping interpolation for gap

In [8]:
segmented_trajectories_df.head()

,BaseDateTime,LAT,LON,SOG,COG,MMSI,SegmentID
0,2022-03-31 00:02:32,27.35372,-94.62546,0.4,228.6,111,0
1,2022-03-31 00:05:35,27.35372,-94.62550,0.6,219.8,111,0
2,2022-03-31 00:08:34,27.35377,-94.62556,0.2,221.7,111,0
3,2022-03-31 00:11:31,27.35380,-94.62557,0.3,105.0,111,0
4,2022-03-31 00:14:33,27.35365,-94.62542,0.3,173.4,111,0


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Add temporal features
def add_temporal_features(data, time_col):
    data[time_col] = pd.to_datetime(data[time_col])
    data['Hour'] = data[time_col].dt.hour
    data['Minute'] = data[time_col].dt.minute
    data['Second'] = data[time_col].dt.second
    data['TimeDelta'] = data[time_col].diff().dt.total_seconds().fillna(0)
    return data


# Filter segments with insufficient rows
def filter_short_segments(data, window_size, prediction_horizon, group_col='SegmentID'):
    min_required_rows = window_size + prediction_horizon
    filtered_data = data.groupby(group_col).filter(lambda x: len(x) >= min_required_rows)
    print(f"Filtered data contains {len(filtered_data)} rows after removing short segments.")
    return filtered_data


# Prepare sequences with temporal features and SegmentID
def prepare_sequences(data, window_size, prediction_horizon,
                      group_col='SegmentID', time_col='BaseDateTime'):
    sequences = []
    unique_segments = sorted(data[group_col].unique())

    for segment in tqdm(unique_segments, desc="Processing Segments", unit="segment"):
        # print('************',segment)
        segment_data = data[data[group_col] == segment].reset_index(drop=True)
        num_rows = len(segment_data)
        # print(f"Segment {segment} has {num_rows} rows.")

        if num_rows < window_size + prediction_horizon:
            # print(f"Skipping segment {segment}: Not enough rows.")
            continue

        for start in range(num_rows - window_size - prediction_horizon + 1):
            x_window = segment_data.iloc[start:start+window_size]
            y_window = segment_data.iloc[start+window_size:start+window_size+prediction_horizon]
            sequences.append({
                'X': {
                    'LAT': x_window['LAT'].tolist(),
                    'LON': x_window['LON'].tolist(),
                    'SOG': x_window['SOG'].tolist(),
                    'Hour': x_window['Hour'].tolist(),
                    'Minute': x_window['Minute'].tolist(),
                    'Second': x_window['Second'].tolist(),
                    'TimeDelta': x_window['TimeDelta'].tolist(),
                    'SegmentID': x_window['SegmentID'].tolist(),
                },
                'Y': {
                    'LAT': y_window['LAT'].tolist(),
                    'LON': y_window['LON'].tolist(),
                    'SOG': y_window['SOG'].tolist(),
                    'Hour': y_window['Hour'].tolist(),
                    'Minute': y_window['Minute'].tolist(),
                    'Second': y_window['Second'].tolist(),
                    'TimeDelta': y_window['TimeDelta'].tolist(),
                    'SegmentID': y_window['SegmentID'].tolist(),
                }
            })
    return sequences


# Split data into train/val/test
def split_train_val_test(data, test_size=0.2, val_size=0.2):
    train_data, temp_data = train_test_split(data, test_size=test_size + val_size, random_state=42, shuffle=True)
    val_data, test_data = train_test_split(temp_data, test_size=test_size / (test_size + val_size), random_state=42, shuffle=True)
    return train_data, val_data, test_data

# def split_train_val_test(data, test_size=0.2, val_size=0.2, group_col='SegmentID'):
#     sorted_segments = sorted(data[group_col].unique())  # Sort SegmentIDs
#     train_segments, temp_segments = train_test_split(sorted_segments, test_size=test_size + val_size, random_state=42)
#     val_segments, test_segments = train_test_split(temp_segments, test_size=test_size / (test_size + val_size), random_state=42)

#     train_data = data[data[group_col].isin(train_segments)]
#     val_data = data[data[group_col].isin(val_segments)]
#     test_data = data[data[group_col].isin(test_segments)]

#     return train_data, val_data, test_data

# def split_train_val_test(data, train_size=0.7, val_size=0.1, test_size=0.2, group_col='SegmentID'):
#     train_data = []
#     val_data = []
#     test_data = []

#     unique_segments = sorted(data[group_col].unique())  # Sort SegmentIDs

#     for segment in unique_segments:
#         segment_data = data[data[group_col] == segment]

#         # Split into train and temp (validation + test)
#         train_segment, temp_segment = train_test_split(
#             segment_data, test_size=(val_size + test_size), random_state=42, shuffle=False
#         )

#         # Further split temp_segment into validation and test
#         val_segment, test_segment = train_test_split(
#             temp_segment, test_size=(test_size / (val_size + test_size)), random_state=42, shuffle=False
#         )

#         # Append splits to their respective lists
#         train_data.append(train_segment)
#         val_data.append(val_segment)
#         test_data.append(test_segment)

#     # Concatenate all segments' splits
#     train_data = pd.concat(train_data).reset_index(drop=True)
#     val_data = pd.concat(val_data).reset_index(drop=True)
#     test_data = pd.concat(test_data).reset_index(drop=True)

#     return train_data, val_data, test_data

# Data preparation pipeline without scaling
def prepare_data_pipeline_no_scaling(data, window_size, prediction_horizon,
                                     test_size=0.2, val_size=0.2):
    # Add temporal features
    print("Adding temporal features...")
    data = add_temporal_features(data, 'BaseDateTime')

    # Filter short segments
    print("Filtering short segments...")
    data = filter_short_segments(data, window_size, prediction_horizon)

    # Split data
    print("Splitting data into train, validation, and test sets...")
    train_data, val_data, test_data = split_train_val_test(data, test_size, val_size)

    # Prepare sequences
    print("Preparing training sequences...")
    train_sequences = prepare_sequences(train_data, window_size, prediction_horizon)

    print("Preparing validation sequences...")
    val_sequences = prepare_sequences(val_data, window_size, prediction_horizon)

    print("Preparing test sequences...")
    test_sequences = prepare_sequences(test_data, window_size, prediction_horizon)

    return train_sequences, val_sequences, test_sequences

# Parameters
window_size = 5
prediction_horizon = 2
test_size = 0.2
val_size = 0.32


# Prepare data
train_sequences, val_sequences, test_sequences = prepare_data_pipeline_no_scaling(
    segmented_trajectories_df, window_size, prediction_horizon, test_size, val_size
)

print(f"Number of training sequences: {len(train_sequences)}")
print(f"Number of validation sequences: {len(val_sequences)}")
print(f"Number of test sequences: {len(test_sequences)}")



Adding temporal features...
Filtering short segments...
Filtered data contains 33190 rows after removing short segments.
Splitting data into train, validation, and test sets...
Preparing training sequences...


Processing Segments: 100%|██████████| 3314/3314 [00:05<00:00, 564.76segment/s]


Preparing validation sequences...


Processing Segments: 100%|██████████| 3241/3241 [00:02<00:00, 1116.84segment/s]


Preparing test sequences...


Processing Segments: 100%|██████████| 2965/2965 [00:02<00:00, 1098.43segment/s]

Number of training sequences: 652
Number of validation sequences: 49
Number of test sequences: 2


In [ ]:
def format_sequences(sequences, input_features, output_features, predicted_seq_len):
    """
    Format sequences for Seq2Seq model.

    Parameters:
    - sequences: List of dictionaries with 'X' and 'Y'.
    - input_features: List of feature names for inputs.
    - output_features: List of feature names for outputs.
    - predicted_seq_len: Length of the predicted sequence.

    Returns:
    - encoder_inputs: NumPy array of shape (num_sequences, input_seq_len, num_features).
    - decoder_inputs: NumPy array of shape (num_sequences, output_seq_len, num_features).
    - targets: NumPy array of shape (num_sequences, output_seq_len, num_targets).
    """
    encoder_inputs = []
    decoder_inputs = []
    targets = []

    for seq in sequences:
        encoder_inputs.append(np.array([seq['X'][feature] for feature in input_features]).T)
        y_data = np.array([seq['Y'][feature] for feature in input_features]).T
        decoder_input = np.zeros((predicted_seq_len, len(input_features)))
        decoder_input[:-1, :] = y_data[:-1, :]  # Fill with previous steps
        decoder_inputs.append(decoder_input)
        targets.append(np.array([seq['Y'][feature] for feature in output_features]).T)  # Full predicted sequence

    return (
        np.array(encoder_inputs),
        np.array(decoder_inputs),
        np.array(targets)
    )


# Define the features for input and output
input_features = ['LAT', 'LON', 'SOG', 'Hour', 'Minute', 'Second', 'TimeDelta']
output_features = ['LAT', 'LON', 'SOG']

predicted_seq_len = 2

# Format sequences for training, validation, and testing
train_enc_inputs, train_dec_inputs, train_targets = format_sequences(train_sequences, input_features, output_features, predicted_seq_len)
val_enc_inputs, val_dec_inputs, val_targets = format_sequences(val_sequences, input_features, output_features, predicted_seq_len)
test_enc_inputs, test_dec_inputs, test_targets = format_sequences(test_sequences, input_features, output_features, predicted_seq_len)

# Print the shapes to verify
print("Training data shapes:")
print(f"Encoder inputs: {train_enc_inputs.shape}, Decoder inputs: {train_dec_inputs.shape}, Targets: {train_targets.shape}")

# Check for NaN or Inf in your data
print("Checking training data for NaN or Inf...")
print(np.isnan(train_enc_inputs).any(), np.isinf(train_enc_inputs).any())
print(np.isnan(train_dec_inputs).any(), np.isinf(train_dec_inputs).any())
print(np.isnan(train_targets).any(), np.isinf(train_targets).any())


Training data shapes:
Encoder inputs: (652, 5, 7), Decoder inputs: (652, 2, 7), Targets: (652, 2, 3)
Checking training data for NaN or Inf...
False False
False False
False False


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, LSTM, Dense, Concatenate, Dropout, MultiHeadAttention,
    Add, LayerNormalization, RepeatVector, Lambda, Conv1D, Bidirectional
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def create_advanced_probabilistic_seq2seq(input_seq_len, output_seq_len, input_dim, output_dim, latent_dim):
    """
    Enhanced Probabilistic Seq2Seq Model for Time Series Forecasting with improvements for handling time series data.
    """
    # Encoder
    encoder_inputs = Input(shape=(input_seq_len, input_dim), name="encoder_inputs")

    # Temporal Convolutional Layers for better feature extraction in time series data
    encoder_conv = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', name="encoder_conv")(encoder_inputs)
    encoder_conv = Dropout(0.3, name="encoder_conv_dropout")(encoder_conv)

    # Bidirectional LSTM for Encoder to capture patterns from both directions
    encoder_lstm = Bidirectional(LSTM(latent_dim, return_sequences=True, return_state=True, name="encoder_lstm"))
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_conv)  # Adjusted unpacking

    # Combine forward and backward states to form initial state for decoder
    state_h = Concatenate()([forward_h, backward_h])
    state_c = Concatenate()([forward_c, backward_c])

    # Multi-Head Attention for Encoder
    multi_head_attention = MultiHeadAttention(num_heads=4, key_dim=latent_dim, name="multi_head_attention")
    encoder_attention = multi_head_attention(encoder_outputs, encoder_outputs)  # Query and value are the same
    encoder_attention = LayerNormalization(name="encoder_attention_norm")(encoder_attention)

    # Decoder
    decoder_inputs = Input(shape=(output_seq_len, input_dim), name="decoder_inputs")

    # Convolutional Layer in Decoder
    decoder_conv = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', name="decoder_conv")(decoder_inputs)
    decoder_conv = Dropout(0.3, name="decoder_conv_dropout")(decoder_conv)

    # Decoder LSTM, using encoder states for initialization
    decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, return_state=True, name="decoder_lstm")
    decoder_outputs, _, _ = decoder_lstm(decoder_conv, initial_state=[state_h, state_c])

    # Ensure that the attention mechanism has compatible shapes (query and value)
    attention_context = multi_head_attention(decoder_outputs, encoder_attention)  # Using encoder attention in decoder
    attention_context = LayerNormalization(name="decoder_attention_norm")(attention_context)
    attention_context = Lambda(lambda x: tf.reduce_mean(x, axis=1), name="summarize_attention")(attention_context)

    # Ensure Attention Context and Decoder Outputs have compatible dimensions
    attention_context_repeated = RepeatVector(output_seq_len)(attention_context)  # Repeat to match output_seq_len
    attention_context_repeated = tf.keras.layers.Reshape((output_seq_len, latent_dim * 2))(attention_context_repeated)

    # Merge Attention Context with Decoder Outputs
    merged_context = Concatenate(axis=-1)([decoder_outputs, attention_context_repeated])

    # Dropout and Output Layers
    dropout = Dropout(0.4, name="decoder_dropout")(merged_context)
    output_mean = Dense(output_dim, activation="linear", kernel_regularizer=tf.keras.regularizers.l2(1e-4),
                        name="mean_output")(dropout)
    output_std = Dense(output_dim, activation="softplus", kernel_regularizer=tf.keras.regularizers.l2(1e-4),
                       name="std_output")(dropout)

    # Combine Outputs
    outputs = Concatenate(name="probabilistic_outputs")([output_mean, output_std])

    # Define the Model
    model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=outputs, name="enhanced_probabilistic_seq2seq_model")

    # Custom Loss Function (Negative Log-Likelihood)
    def nll_loss(y_true, y_pred):
        mean = y_pred[..., :output_dim]
        std = y_pred[..., output_dim:]
        dist = tf.compat.v1.distributions.Normal(loc=mean, scale=std)
        return -tf.reduce_mean(dist.log_prob(y_true))

    # Probabilistic Evaluation Metric: CRPS
    def crps_metric(y_true, y_pred):
        mean = y_pred[..., :output_dim]
        std = y_pred[..., output_dim:]
        return tf.reduce_mean((y_true - mean) ** 2 / (2 * std ** 2) + tf.math.log(std))

    # Adjusted R² Metric
    def adjusted_r2_score(y_true, y_pred):
        mean = y_pred[..., :output_dim]
        ss_res = tf.reduce_sum(tf.square(y_true - mean), axis=-1)
        ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true, axis=-1, keepdims=True)), axis=-1)
        r2 = 1 - (ss_res / (ss_tot + tf.keras.backend.epsilon()))

        # Number of samples (batch size)
        n = tf.cast(tf.shape(y_true)[0], tf.float32)

        # Number of predictors (features)
        p = tf.cast(output_dim, tf.float32)

        # Adjusted R²
        adjusted_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1 + tf.keras.backend.epsilon())
        return adjusted_r2

    # Optimizer
    optimizer = Adam(learning_rate=0.01)

    # Compile the Model
    model.compile(optimizer=optimizer, loss=nll_loss, metrics=[crps_metric, adjusted_r2_score])

    return model

# Create and Compile the Model
input_seq_len = 5
output_seq_len = 2
input_dim = 7
output_dim = 3
latent_dim = 64

model = create_advanced_probabilistic_seq2seq(input_seq_len, output_seq_len, input_dim, output_dim, latent_dim)

# Summary
model.summary()


Model: "enhanced_probabilistic_seq2seq_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, 5, 7)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_conv (Conv1D)     │ (None, 5, 64)          │          1,408 │ encoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, 2, 7)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_conv_dropout      │ (None, 5, 64)          │              0 │ encoder_conv[0][0]     │
│ (Dropout)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_conv (Conv1D)     │ (None, 2, 64)          │          1,408 │ decoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ [(None, 5, 128),       │         66,048 │ encoder_conv_dropout[… │
│ (Bidirectional)           │ (None, 64), (None,     │                │                        │
│                           │ 64), (None, 64),       │                │                        │
│                           │ (None, 64)]            │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_conv_dropout      │ (None, 2, 64)          │              0 │ decoder_conv[0][0]     │
│ (Dropout)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 128)            │              0 │ bidirectional[0][1],   │
│                           │                        │                │ bidirectional[0][3]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 128)            │              0 │ bidirectional[0][2],   │
│ (Concatenate)             │                        │                │ bidirectional[0][4]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 2, 128)         │        131,968 │ bidirectional[0][0],   │
│ (MultiHeadAttention)      │                        │                │ bidirectional[0][0],   │
│                           │                        │                │ decoder_lstm[0][0],    │
│                           │                        │                │ encoder_attention_nor… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_lstm (LSTM)       │ [(None, 2, 128),       │         98,816 │ decoder_conv_dropout[… │
│                           │ (None, 128), (None,    │                │ concatenate[0][0],     │
│                           │ 128)]                  │                │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_attention_norm    │ (None, 5, 128)         │            256 │ multi_head_attention[… │
│ (LayerNormalization)      │                        │                │                        │
├──────────────────────

 Total params: 301,702 (1.15 MB)

 Trainable params: 301,702 (1.15 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Hyperparameters
input_seq_len = 5  # Input sequence length
output_seq_len = 2  # Output sequence length
latent_dim = 64  # Number of LSTM units
input_dim = len(input_features)  # Number of input features
output_dim = len(output_features)  # Number of output features

# Create the model
model = create_advanced_probabilistic_seq2seq(input_seq_len, output_seq_len, input_dim, output_dim, latent_dim)

# Callbacks
lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, verbose=1)
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True, verbose=1)

# Train the model
print("Training the model...")
history = model.fit(
    [train_enc_inputs, train_dec_inputs],
    train_targets,
    validation_data=([val_enc_inputs, val_dec_inputs], val_targets),
    batch_size=32,
    epochs=45,
    callbacks=[lr_scheduler, early_stopping],
    verbose=1
)

# Save the Model
model.save_weights("prob_attention_lstm.weights.h5")
print("Model saved successfully.")


Training the model...
Epoch 1/45


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


21/21 ━━━━━━━━━━━━━━━━━━━━ 12s 83ms/step - adjusted_r2_score: -0.2528 - crps_metric: 13936.1787 - loss: 13948.1631 - val_adjusted_r2_score: -0.2485 - val_crps_metric: 15.5309 - val_loss: 17.1769 - learning_rate: 0.0100
Epoch 2/45
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - adjusted_r2_score: -0.1969 - crps_metric: 14.9011 - loss: 15.8272 - val_adjusted_r2_score: -0.2120 - val_crps_metric: 12.7936 - val_loss: 14.2180 - learning_rate: 0.0100
Epoch 3/45
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - adjusted_r2_score: -0.1661 - crps_metric: 12.9156 - loss: 13.8394 - val_adjusted_r2_score: -0.1825 - val_crps_metric: 11.6215 - val_loss: 12.9787 - learning_rate: 0.0100
Epoch 4/45
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - adjusted_r2_score: -0.1346 - crps_metric: 11.9100 - loss: 12.8381 - val_adjusted_r2_score: -0.1387 - val_crps_metric: 10.2516 - val_loss: 11.5541 - learning_rate: 0.0100
Epoch 5/45
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - adjusted_r2_score: -0.0915 - crps_metric: 10.4440 - loss: 1

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate, Dropout, Attention, Bidirectional, Add, RepeatVector, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from scipy.signal import savgol_filter

# Preprocess the data for inference
def preprocess_data_inference(df):
    if 'BaseDateTime' not in df.columns:
        raise ValueError("Data must include 'BaseDateTime'")
    df['BaseDateTime'] = pd.to_datetime(df['BaseDateTime'])
    df['Hour'] = df['BaseDateTime'].dt.hour
    df['Minute'] = df['BaseDateTime'].dt.minute
    df['Second'] = df['BaseDateTime'].dt.second
    df['TimeDelta'] = df['BaseDateTime'].diff().dt.total_seconds().fillna(0)
    return df

# Handle variable sequence lengths
def pad_sequence(data, desired_length, pad_value=0):
    if len(data) < desired_length:
        padding = np.full((desired_length - len(data), data.shape[1]), pad_value)
        data = np.vstack([padding, data])
    return data

# Prepare encoder and decoder inputs for inference
def prepare_inputs_inference(data, window_size, input_features):
    if len(data) < window_size:
        encoder_input = pad_sequence(data[input_features].values, window_size)
    else:
        encoder_input = data[input_features].iloc[-window_size:].values

    encoder_input = encoder_input.reshape(1, window_size, -1)

    decoder_input = np.zeros((1, output_seq_len, len(input_features)))
    decoder_input[0, 0, :] = data[input_features].iloc[-1:].values

    return encoder_input, decoder_input

# Predict future steps with constraints and corrections
def predict_future_steps_with_constraints(
    model, encoder_input, decoder_input, steps, temporal_features,
    dynamic_alpha=True, velocity_limit=0.60, acceleration_limit=0.56, alpha=0.7
):
    # alpha parameter determines the weight given to the model's predictions versus the previous observations during the prediction adjustment step
    predictions = []
    lower_bounds = []
    upper_bounds = []

    current_encoder_input = encoder_input.copy()
    current_decoder_input = decoder_input.copy()

    for step in range(steps):
        output = model.predict([current_encoder_input, current_decoder_input], verbose=0)
        mean = output[..., :len(output_features)]
        std = output[..., len(output_features):]
        prediction = mean[:, 0, :]

        if dynamic_alpha:
            current_alpha = np.clip(1 - np.mean(std), 0.5, 0.9)
        else:
            current_alpha = alpha

        adjusted_prediction = (
            prediction * current_alpha + current_encoder_input[:, -1, :len(output_features)] * (1 - current_alpha)
        )

        if len(predictions) > 0:
            prev_prediction = predictions[-1]
            velocity = adjusted_prediction - prev_prediction
            velocity = np.clip(velocity, -velocity_limit, velocity_limit)

            if len(predictions) > 1:
                acceleration = velocity - (prev_prediction - predictions[-2])
                acceleration = np.clip(acceleration, -acceleration_limit, acceleration_limit)
                adjusted_prediction = prev_prediction + acceleration
            else:
                adjusted_prediction = prev_prediction + velocity

        adjusted_prediction[:, 0] = np.clip(adjusted_prediction[:, 0], -90, 90)
        adjusted_prediction[:, 1] = np.clip(adjusted_prediction[:, 1], -180, 180)

        lower_bound = adjusted_prediction - 1.96 * std[:, 0, :]
        upper_bound = adjusted_prediction + 1.96 * std[:, 0, :]

        predictions.append(adjusted_prediction)
        lower_bounds.append(lower_bound)
        upper_bounds.append(upper_bound)

        temporal_features['BaseDateTime'] += pd.Timedelta(seconds=60)
        base_datetime = temporal_features['BaseDateTime'].iloc[0]
        temporal_features['Hour'] = base_datetime.hour
        temporal_features['Minute'] = base_datetime.minute
        temporal_features['Second'] = base_datetime.second
        temporal_features['TimeDelta'] = 60

        predicted_with_temporal = np.hstack([
            adjusted_prediction,
            temporal_features[['Hour', 'Minute', 'Second', 'TimeDelta']].values
        ])
        predicted_with_temporal = predicted_with_temporal.reshape(1, 1, -1)

        current_decoder_input = np.concatenate([current_decoder_input[:, 1:, :], predicted_with_temporal], axis=1)
        current_encoder_input = np.concatenate([current_encoder_input[:, 1:, :], predicted_with_temporal], axis=1)

    if len(predictions) >= 5:
        smoothed_predictions = savgol_filter(
            np.array(predictions).squeeze(axis=1),
            window_length=min(len(predictions), 5),
            polyorder=2,
            axis=0
        )
    else:
        smoothed_predictions = np.array(predictions).squeeze(axis=1)

    return smoothed_predictions, lower_bounds, upper_bounds

# Parameters
input_seq_len = 5
output_seq_len = 2
latent_dim = 64
input_features = ['LAT', 'LON', 'SOG', 'Hour', 'Minute', 'Second', 'TimeDelta']
output_features = ['LAT', 'LON', 'SOG']
window_size = input_seq_len

# Load preprocessed data (replace with actual data)
input_df = pd.read_csv('/content/segmented_trajectories.csv',nrows=5)
data = preprocess_data_inference(input_df)
print("Observations:")
print(data)
data = preprocess_data_inference(data)

# Prepare inputs for inference
encoder_input, decoder_input = prepare_inputs_inference(data, window_size, input_features)

# Define and load the model
model = create_advanced_probabilistic_seq2seq(input_seq_len, output_seq_len, len(input_features), len(output_features), latent_dim)
model.load_weights("prob_attention_lstm.weights.h5")
print("Model weights loaded successfully.")

# Predict future steps with drift reduction
steps_to_predict = 3
temporal_features = data.iloc[-1:]
predictions, lower_bounds, upper_bounds = predict_future_steps_with_constraints(
    model, encoder_input, decoder_input, steps_to_predict, temporal_features, alpha=0.7
)

# Output predictions
print("Predicted future steps (LAT, LON, SOG):")
print(predictions)
# print("Confidence intervals (lower bounds, upper bounds):")
# print(lower_bounds)
# print(upper_bounds)


Observations:
         BaseDateTime       LAT       LON  SOG    COG  MMSI  SegmentID  Hour  \
0 2022-03-31 00:02:32  27.35372 -94.62546  0.4  228.6   111          0     0   
1 2022-03-31 00:05:35  27.35372 -94.62550  0.6  219.8   111          0     0   
2 2022-03-31 00:08:34  27.35377 -94.62556  0.2  221.7   111          0     0   
3 2022-03-31 00:11:31  27.35380 -94.62557  0.3  105.0   111          0     0   
4 2022-03-31 00:14:33  27.35365 -94.62542  0.3  173.4   111          0     0   

   Minute  Second  TimeDelta  
0       2      32        0.0  
1       5      35      183.0  
2       8      34      179.0  
3      11      31      177.0  
4      14      33      182.0  
Model weights loaded successfully.
Predicted future steps (LAT, LON, SOG):
[[ 29.10517831 -76.24769039   2.30781116]
 [ 28.86086152 -75.64769039   2.90781116]
 [ 29.42086152 -75.64769039   2.90781116]]


### Pred - v2

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate, Dropout, Attention, Bidirectional
from tensorflow.keras.models import Model
from geopy.distance import geodesic
from scipy.signal import savgol_filter


# Calculate distances and headings between observations
def calculate_distance_and_heading(df):
    distances = []
    headings = []
    for i in range(1, len(df)):
        prev_point = (df.iloc[i - 1]['LAT'], df.iloc[i - 1]['LON'])
        curr_point = (df.iloc[i]['LAT'], df.iloc[i]['LON'])

        # Calculate distance
        distance = geodesic(prev_point, curr_point).meters
        distances.append(distance)

        # Calculate heading
        heading = np.arctan2(
            np.sin(np.radians(curr_point[1] - prev_point[1])) * np.cos(np.radians(curr_point[0])),
            np.cos(np.radians(prev_point[0])) * np.sin(np.radians(curr_point[0])) -
            np.sin(np.radians(prev_point[0])) * np.cos(np.radians(curr_point[0])) *
            np.cos(np.radians(curr_point[1] - prev_point[1]))
        )
        headings.append(np.degrees(heading))

    distances.insert(0, 0)  # No distance for the first observation
    headings.insert(0, 0)  # No heading for the first observation

    df['Distance'] = distances
    df['Heading'] = headings
    return df


# Preprocess the data for inference
def preprocess_data_inference(df):
    if 'BaseDateTime' not in df.columns:
        raise ValueError("Data must include 'BaseDateTime'")
    df['BaseDateTime'] = pd.to_datetime(df['BaseDateTime'])
    df['Hour'] = df['BaseDateTime'].dt.hour
    df['Minute'] = df['BaseDateTime'].dt.minute
    df['Second'] = df['BaseDateTime'].dt.second
    df['TimeDelta'] = df['BaseDateTime'].diff().dt.total_seconds().fillna(0)

    # Calculate distance and heading
    df = calculate_distance_and_heading(df)

    # Apply distance and heading weights
    weight_distance = 0.5
    weight_heading = 0.5

    df['Weighted_LAT'] = df['LAT'] + weight_distance * (df['Distance'] / df['Distance'].max())
    df['Weighted_LON'] = df['LON'] + weight_heading * (df['Heading'] / 180)
    return df


# Prepare encoder and decoder inputs for inference
def prepare_inputs_inference(data, window_size, input_features):
    if len(data) < window_size:
        encoder_input = pad_sequence(data[input_features].values, window_size)
    else:
        encoder_input = data[input_features].iloc[-window_size:].values

    encoder_input = encoder_input.reshape(1, window_size, -1)

    decoder_input = np.zeros((1, output_seq_len, len(input_features)))
    decoder_input[0, 0, :] = data[input_features].iloc[-1:].values

    return encoder_input, decoder_input

# Predict future steps with constraints and corrections
def predict_future_steps_with_constraints(
    model, encoder_input, decoder_input, steps, temporal_features,
    dynamic_alpha=True, velocity_limit=0.0, acceleration_limit=0.0, alpha=0.1
):
    predictions = []
    lower_bounds = []
    upper_bounds = []

    current_encoder_input = encoder_input.copy()
    current_decoder_input = decoder_input.copy()

    for step in range(steps):
        output = model.predict([current_encoder_input, current_decoder_input], verbose=0)
        mean = output[..., :len(output_features)]
        std = output[..., len(output_features):]
        prediction = mean[:, 0, :]

        if dynamic_alpha:
            current_alpha = np.clip(1 - np.mean(std), 0.5, 0.9)
        else:
            current_alpha = alpha

        adjusted_prediction = (
            prediction * current_alpha + current_encoder_input[:, -1, :len(output_features)] * (1 - current_alpha)
        )

        if len(predictions) > 0:
            prev_prediction = predictions[-1]
            velocity = adjusted_prediction - prev_prediction
            velocity = np.clip(velocity, -velocity_limit, velocity_limit)

            if len(predictions) > 1:
                acceleration = velocity - (prev_prediction - predictions[-2])
                acceleration = np.clip(acceleration, -acceleration_limit, acceleration_limit)
                adjusted_prediction = prev_prediction + acceleration
            else:
                adjusted_prediction = prev_prediction + velocity

        adjusted_prediction[:, 0] = np.clip(adjusted_prediction[:, 0], -90, 90)
        adjusted_prediction[:, 1] = np.clip(adjusted_prediction[:, 1], -180, 180)

        lower_bound = adjusted_prediction - 1.96 * std[:, 0, :]
        upper_bound = adjusted_prediction + 1.96 * std[:, 0, :]

        predictions.append(adjusted_prediction)
        lower_bounds.append(lower_bound)
        upper_bounds.append(upper_bound)

        temporal_features['BaseDateTime'] += pd.Timedelta(seconds=60)
        base_datetime = temporal_features['BaseDateTime'].iloc[0]
        temporal_features['Hour'] = base_datetime.hour
        temporal_features['Minute'] = base_datetime.minute
        temporal_features['Second'] = base_datetime.second
        temporal_features['TimeDelta'] = 60

        predicted_with_temporal = np.hstack([
            adjusted_prediction,
            temporal_features[['Hour', 'Minute', 'Second', 'TimeDelta']].values
        ])
        predicted_with_temporal = predicted_with_temporal.reshape(1, 1, -1)

        current_decoder_input = np.concatenate([current_decoder_input[:, 1:, :], predicted_with_temporal], axis=1)
        current_encoder_input = np.concatenate([current_encoder_input[:, 1:, :], predicted_with_temporal], axis=1)

    if len(predictions) >= 5:
        smoothed_predictions = savgol_filter(
            np.array(predictions).squeeze(axis=1),
            window_length=min(len(predictions), 5),
            polyorder=2,
            axis=0
        )
    else:
        smoothed_predictions = np.array(predictions).squeeze(axis=1)

    return smoothed_predictions, lower_bounds, upper_bounds


# Main execution
input_seq_len = 10
output_seq_len = 5
latent_dim = 64
input_features = ['LAT', 'LON', 'SOG', 'Hour', 'Minute', 'Second', 'TimeDelta']
output_features = ['LAT', 'LON', 'SOG']

input_df = pd.read_csv('/content/segmented_trajectories.csv',nrows=5)
print("Observation data")
print(df)
data = preprocess_data_inference(df)

# Prepare inputs for inference
encoder_input, decoder_input = prepare_inputs_inference(data, input_seq_len, input_features)

# Define and load the model
model = create_advanced_probabilistic_seq2seq(input_seq_len, output_seq_len, len(input_features), len(output_features), latent_dim)
# Ensure model weights are already trained and available as "model_weights.h5"
# model.load_weights("model_weights.h5")

# Predict future steps
steps_to_predict = 3
temporal_features = data.iloc[-1:].copy()
predictions, lower_bounds, upper_bounds = predict_future_steps_with_constraints(
    model, encoder_input, decoder_input, steps_to_predict, temporal_features, alpha=0.7
)

# Output predictions
print("Predicted future steps (LAT, LON, SOG):")
print(predictions)


Observation data
              BaseDateTime       MMSI         IMO       LAT        LON   SOG  \
0      2022-03-31T00:00:01  671226100  IMO9221322  25.77626  -80.20320   3.2   
1      2022-03-31T00:00:06  367452810  IMO9602344  29.32824  -94.77391   2.6   
2      2022-03-31T00:00:09  366919770  IMO9253583  48.74428 -122.49504   0.0   
3      2022-03-31T00:00:00  311000966  IMO8916607  18.55833  -66.47910  17.5   
4      2022-03-31T00:00:01  219028420  IMO9411305  28.66703  -93.59339   9.5   
...                    ...        ...         ...       ...        ...   ...   
29995  2022-03-31T00:33:43  366866930  IMO9258806  48.18500 -123.25926  13.1   
29996  2022-03-31T00:34:18  303104000  IMO7816551  27.93877  -82.44400   0.0   
29997  2022-03-31T00:29:53  367622660  IMO9207948  29.12027  -90.21229   0.0   
29998  2022-03-31T00:29:55  316003600  IMO8020018  49.31200 -123.10683   0.0   
29999  2022-03-31T00:29:59  367405770  IMO9529736  30.39321  -81.40451   0.1   

         COG  
0      